# scraping-github-topics-repositories-final

* introduction to web scraping
* Introduction to Github and the problem statement
* tools used include (Python, requests, Beautiful Soup, Pandas)

Outline
* we're going to scrape https://github.com/topics
* Get the list of the topics. For each topic, topic title, topic page URL and topic descripyion
* For each topic, get the top 25 repos in the topic page
* For each repository we'll grab the repo name, username, stars and repo URL
* For each topic we'll create a CSV file in the following format


Repo Name,Username,Stars,Repo URL three.js,mrdoob,69700,https://github.com/mrdoob/three.js libgdx,libgdx,18300,https://github.com/libgdx/libgdx

!pip install  requests --upgrade --quiet

In [28]:
!pip install jovian --upgrade --quiet

In [ ]:
import jovian

## Scrape the list of topic from Github

* use requests to download the page
* use Beautiful Soup to parse and extract information
* convert to a Pandas dataframe

In [10]:
import requests
from bs4 import BeautifulSoup
def get_topics_page():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code !=200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [11]:
doc = get_topics_page()

In [12]:
doc.find('a')

<a class="px-2 py-4 color-bg-accent-emphasis color-fg-on-emphasis show-on-focus js-skip-to-content" href="#start-of-content">Skip to content</a>

In [13]:
def get_topic_titles(doc):
    selection_class = 'f3 lh-condensed mb-0 mt-1 Link--primary'
    topic_title_tags = doc.find_all('p', {'class': selection_class})
    topic_titles = []
    for tag in topic_title_tags:
        topic_titles.append(tag.text)
    return topic_titles


In [14]:
def get_topic_descs(docs):
    desc_selector = 'f5 color-fg-muted mb-0 mt-1'
    topic_desc_tags = doc.find_all('p', {'class': desc_selector})
    topic_descs =  []
    for tag in topic_desc_tags:
        topic_descs.append(tag.text.strip())
    return topic_descs


In [15]:
def get_topic_urls(doc):
    topic_link_tags = doc.find_all('a', {'class': 'd-flex no-underline'})
    topic_urls = []
    base_url  = 'https://github.com'
    for tag in topic_link_tags:
        topic_urls.append(base_url + tag['href'])
    return topic_urls


In [ ]:
def scrape_topics():
    topics_url = 'https://github.com/topics'
    # response = requests = requests.get(topics_url)
    if response.status_code !=200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'htmlmparser')
    topics_dict = {
        'title' : get_topic_titles(doc),
        'description': get_topic_descs(doc),
        'url': get_topic_urls(doc)
    }
    return pd.DataFrame(topics_dict)


In [16]:
titles = get_topic_titles(doc)

In [17]:
len(titles)

30

In [19]:
titles[:5]

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']

## get the top 25 repos in the topic page

In [22]:
def get_topics_page():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code !=200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [24]:
#doc = get_topic_page('https://github.com/topics/3d')

In [25]:
def get_repo_info(h1_tag, star_tag):
    #returns all the required info about  a repo
    a_tags = h1_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.stri())
    return username, repo_name, stars, repo_url


In [26]:
def get_topic_repos(topic_doc):
    #response = requests = requests.get(topics_url)
    if response.status_code !=200:
        raise Exception('Failed to load page {}'.format(topic_url))
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    h1_selection_class = 'f3 color-text-secondary text-normal lh-condensed'
    repo_tags = topic_doc.find_all('h1', {'class': h1_selection_class})

    star_tags = topic_doc.find_all('a', {'class': 'social-count float-none'})

    topic_repos_dict = { 
        'username': [], 
        'repo_name': [],
        'stars': [],
        'repo_url': []
    }



    # Get repo info
    for i in range(len(repo_tags)):
        repo_info = get_repo_info(repo_tags[i], star_tags[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['stars'].append(repo_info[2])
        topic_repos_dict['repo_url'].append(repo_info[3])


    return pd.DataFrame(topic_repos_dict)
  

In [27]:

def scrape_topic(topic_url, path):
    if os.path.exists(path):
        print("The file {} already exists. Skipping...".format(path))
        return
    topic_df = get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(path, index = None)
    

In [29]:
# Execute this to save new versions of the notebook
jovian.commit(project="scraping-github-topics-repositories-final")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "lydia02/scraping-github-topics-repositories-final" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/lydia02/scraping-github-topics-repositories-final


'https://jovian.ai/lydia02/scraping-github-topics-repositories-final'

## putting all together

In [30]:
def scrape_topics_repos():
    print('Scraping list of topics')
    topics_df = scrape_topics()
    
    os.makedirs('data', exist_ok = True)
    for index, row in topics_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['title']))
        scrape_topic(row['url'], 'data/{}.csv'.format(row['title']))

In [31]:
scrape_topics_repos

<function __main__.scrape_topics_repos()>